# Light Pollution Data & EDA


1. Data Provenance 

    a. Where is the data from?
        Open night time lights by world bank (Uses both DMSP and VIIR data)
        https://worldbank.github.io/OpenNightLights/welcome.html

    
    b. Is it reliable? (partially subjective)
        Yes, true satellite and sensor data to get radiance/night lights (can use google earth engine)
    
    
2. Time measures

    a. Time range of the dataset
        2014-01-01T00:00:00Z - 2022-05-01T00:00:00
    
    b. Granularity (Weekly, Monthly, Aperiodic, Daily, Timestamp)
         Monthly aggregates available, satellite collects data daily. I think we don't need daily granularity for light pollution. Daily granularity has too much noise due to clouds especially near the equator
    
    
3. Location

    a. Is it available for the UK, Brazil/India?
        Yes
    
    b. Granularity (is it a pin point location?, Rounded coordinates?...)
        750 m^2 area is the minimal resolution
    
    c. Format (shape files, lat long, geographical code)
        can use any of the above
    
    
4. Data Gaps and size

    a. Size of the data
        Entire world is around 250TB. We only need monthly composites for some regions ans some years
        
    b. Available columns and columns of interest
        Radiance, avg_radiance, SOL (sum of lights),DNB radiance, I5 (LWIR) radiance, lunar illuminance, sample position within DNB scan, quality bitflag or “vflag” grid with on/off states for these fields:, daytime/nighttime/near-terminator, zero lunar illuminance, viirs cloud mask, nightfire detection, lightning, high energy particle hit,stray light affected/corrected


    c. Missing data (amount and years/locations missing)
        NA but resolution is the cause of missing data
    
    d. Nulls or Nans
        NA
    
    e. Primary key
        latitude longitude?
    
    f. Columns to join on
        latitude, longitude, date
    
    g. Any additional fields
        Cloud cover resolution, scale factor, clipping and country boundaries,
        Quarterly data updates
    
    
5. Ease of access

    a. Is there a free to use license?s
        Yes, open access
    
    b. How is data accessed? (API, download)
        Java or python API calls. Files available for download but too large
    
    c. Is the documentation straightforward?
        Yes
    
    d. Any limits on how much data we can download?
       No

## Light Pollution

* Glare – excessive brightness that causes visual discomfort
* Skyglow – brightening of the night sky over inhabited areas
* Light trespass – light falling where it is not intended or needed
* Clutter – bright, confusing and excessive groupings of light sources

In [1]:
import pandas as pd
import geemap, ee
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [2]:
center_lat = -3.497341
center_lon = 38.642394

zoomlevel=15


# get December image, we're using the "avg_rad" band
viirs2020_01 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2022-01-01","2022-01-31").select('avg_rad').median()


# initialize our map
map1 = geemap.Map()
map1.add_basemap('SATELLITE')
map1.addLayer(viirs2020_01, {}, "VIIRS-DNB")

map1.addLayerControl()
map1


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
lat = center_lat
lon = center_lon
aoi = ee.Geometry.Point([lon, lat]).buffer(20000);
viirs2020_01_clipped = viirs2020_01.clip(aoi)

map2 = geemap.Map(center=[lat, lon],zoom=10)
map2.add_basemap('SATELLITE')
map2.addLayer(viirs2020_01_clipped, {}, "VIIRS-DNB- Greater LA Dec 2019")
map2.addLayerControl()
map2

Map(center=[-3.497341, 38.642394], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [4]:
def get_avg_radiance(lat, lon, date, buffer=20000, scale=100, geometry = None):
    """
    buffer (int): value in meters of radius around center lat lon
    """
    yy = date.split('-')[0] 
    mm = date.split('-')[1] 
    start = yy + '-' + mm + '-01' ##get data for the month
    
    viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").\
    filterDate(start).select('avg_rad').median()
    aoi = ee.Geometry.Point([lon, lat]).buffer(buffer);
    viirs_clipped = viirs.clip(aoi)
    
    avg_rad = viirs_clipped.reduceRegion(reducer=ee.Reducer.mean(),
                                     geometry = None,
                                     scale=scale,
                                     maxPixels=1e9).get('avg_rad')
    
    return avg_rad.getInfo()


In [5]:
get_avg_radiance(lat=center_lat, lon=center_lon, date='2022-01-01', buffer=20000, scale=100)

0.2997921929806698

In [6]:
brazil0 = ee.Feature(ee.FeatureCollection("FAO/GAUL/2015/level0").\
                     filter(ee.Filter.eq('ADM0_NAME', 'Brazil')).first()).geometry()
india0 = ee.Feature(ee.FeatureCollection("FAO/GAUL/2015/level0").\
                     filter(ee.Filter.eq('ADM0_NAME', 'India')).first()).geometry()
uk0 = ee.Feature(ee.FeatureCollection("FAO/GAUL/2015/level0").\
                     filter(ee.Filter.eq('ADM0_NAME', 'U.K. of Great Britain and Northern Ireland')).first()).geometry()


In [7]:
viirs2020_01_clipped = viirs2020_01.clip(uk0)

map2 = geemap.Map(center=[lat, lon],zoom=7)
map2.add_basemap('SATELLITE')
map2.addLayer(viirs2020_01_clipped, {}, "VIIRS-DNB- Greater LA Dec 2019")
map2.addLayerControl()
map2

Map(center=[-3.497341, 38.642394], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…